# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [5]:
price_df = pd.read_csv('price_data (1).csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [6]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

In [7]:
"""
Feature Engineering for Client Churn Prediction
This script creates comprehensive features based on:
1. Price sensitivity and variations
2. Consumption patterns and trends
3. Contract and temporal features
4. Customer behavior metrics
"""

import pandas as pd
import numpy as np
from scipy import stats

# 1. IMPORT PACKAGES AND LOAD DATA
print("\nLoading data...")
df = pd.read_csv('./clean_data_after_eda.csv')
price_df = pd.read_csv('price_data (1).csv')

# Convert date columns
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')

print(f"Client data shape: {df.shape}")
print(f"Price data shape: {price_df.shape}")

# 2. PRICE-BASED FEATURES
print("\nCREATING PRICE-BASED FEATURES\n")

monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean', 
    'price_off_peak_fix': 'mean',
    'price_peak_var': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_var': 'mean',
    'price_mid_peak_fix': 'mean'
}).reset_index()

jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

diff = pd.merge(
    dec_prices.rename(columns={
        'price_off_peak_var': 'dec_off_peak_var',
        'price_off_peak_fix': 'dec_off_peak_fix',
        'price_peak_var': 'dec_peak_var',
        'price_peak_fix': 'dec_peak_fix'
    }), 
    jan_prices.drop(columns='price_date'), 
    on='id'
)

diff['offpeak_diff_dec_jan_energy'] = diff['dec_off_peak_var'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_jan_power'] = diff['dec_off_peak_fix'] - diff['price_off_peak_fix']
diff['peak_diff_dec_jan_energy'] = diff['dec_peak_var'] - diff['price_peak_var']
diff['peak_diff_dec_jan_power'] = diff['dec_peak_fix'] - diff['price_peak_fix']

price_stats = price_df.groupby('id').agg({
    'price_off_peak_var': ['mean', 'std', 'min', 'max'],
    'price_peak_var': ['mean', 'std', 'min', 'max'],
    'price_off_peak_fix': ['mean', 'std', 'min', 'max'],
    'price_peak_fix': ['mean', 'std', 'min', 'max']
}).reset_index()

price_stats.columns = ['_'.join(col).strip('_') for col in price_stats.columns.values]
price_stats.columns = ['id'] + [f'price_{col}' for col in price_stats.columns[1:]]

price_stats['price_off_peak_var_range'] = price_stats['price_price_off_peak_var_max'] - price_stats['price_price_off_peak_var_min']
price_stats['price_peak_var_range'] = price_stats['price_price_peak_var_max'] - price_stats['price_price_peak_var_min']
price_stats['price_off_peak_fix_range'] = price_stats['price_price_off_peak_fix_max'] - price_stats['price_price_off_peak_fix_min']

price_df_sorted = price_df.sort_values(['id', 'price_date'])
first_3_months = price_df_sorted.groupby('id').head(3).groupby('id').agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_off_peak_fix': 'mean'
}).reset_index()
first_3_months.columns = ['id', 'first3m_off_peak_var', 'first3m_peak_var', 'first3m_off_peak_fix']

last_3_months = price_df_sorted.groupby('id').tail(3).groupby('id').agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_off_peak_fix': 'mean'
}).reset_index()
last_3_months.columns = ['id', 'last3m_off_peak_var', 'last3m_peak_var', 'last3m_off_peak_fix']

price_trend = pd.merge(first_3_months, last_3_months, on='id')
price_trend['price_trend_off_peak_var'] = price_trend['last3m_off_peak_var'] - price_trend['first3m_off_peak_var']
price_trend['price_trend_peak_var'] = price_trend['last3m_peak_var'] - price_trend['first3m_peak_var']
price_trend['price_trend_off_peak_fix'] = price_trend['last3m_off_peak_fix'] - price_trend['first3m_off_peak_fix']

price_changes = price_df.groupby('id').apply(
    lambda x: (x.sort_values('price_date')['price_off_peak_var'].diff() != 0).sum()
).reset_index()
price_changes.columns = ['id', 'num_price_changes_off_peak']

print(f"Created price-based features")

# 3. CONSUMPTION-BASED FEATURES
print("\nCREATING CONSUMPTION-BASED FEATURES\n")

df['cons_12m_per_pow_max'] = df['cons_12m'] / (df['pow_max'] + 1)
df['cons_gas_ratio'] = df['cons_gas_12m'] / (df['cons_12m'] + 1)
df['actual_vs_forecast'] = df['cons_12m'] / (df['forecast_cons_12m'] + 1)
df['recent_consumption_ratio'] = df['cons_last_month'] / (df['cons_12m'] / 12 + 1)

df['is_low_consumer'] = (df['cons_12m'] < df['cons_12m'].quantile(0.25)).astype(int)
df['is_high_consumer'] = (df['cons_12m'] > df['cons_12m'].quantile(0.75)).astype(int)
df['has_recent_consumption'] = (df['cons_last_month'] > 0).astype(int)
df['total_energy_consumption'] = df['cons_12m'] + df['cons_gas_12m']

print(f"Created consumption-based features")

# 4. MARGIN AND PROFITABILITY FEATURES
print("\nCREATING MARGIN AND PROFITABILITY FEATURES\n")

df['margin_per_kwh'] = df['net_margin'] / (df['cons_12m'] + 1)
df['margin_efficiency'] = df['net_margin'] / (df['pow_max'] + 1)

df['is_high_margin_client'] = (df['net_margin'] > df['net_margin'].quantile(0.75)).astype(int)
df['is_low_margin_client'] = (df['net_margin'] < df['net_margin'].quantile(0.25)).astype(int)

df['margin_to_consumption_ratio'] = df['net_margin'] / (df['cons_12m'] + 1)
df['gross_to_net_margin_ratio'] = df['margin_gross_pow_ele'] / (df['margin_net_pow_ele'] + 1)

print(f"Created margin and profitability features")

# 5. TEMPORAL/DATE-BASED FEATURES
print("\nCREATING TEMPORAL FEATURES\n")

df['contract_length_days'] = (df['date_end'] - df['date_activ']).dt.days
df['contract_length_years'] = df['contract_length_days'] / 365.25
df['days_to_end'] = (df['date_end'] - pd.Timestamp('2016-01-01')).dt.days
df['months_since_modification'] = ((pd.Timestamp('2016-01-01') - df['date_modif_prod']).dt.days / 30.44).round()
df['months_to_renewal'] = ((df['date_renewal'] - pd.Timestamp('2016-01-01')).dt.days / 30.44).round()

df['activation_month'] = df['date_activ'].dt.month
df['activation_year'] = df['date_activ'].dt.year
df['activation_quarter'] = df['date_activ'].dt.quarter
df['is_summer_activation'] = df['activation_month'].isin([6, 7, 8]).astype(int)
df['is_winter_activation'] = df['activation_month'].isin([12, 1, 2]).astype(int)

df['has_modified_contract'] = (df['date_activ'] != df['date_modif_prod']).astype(int)
df['modification_to_renewal_days'] = (df['date_renewal'] - df['date_modif_prod']).dt.days

df['tenure_category'] = pd.cut(df['num_years_antig'], bins=[0, 3, 5, 8, 15], labels=['new', 'established', 'loyal', 'veteran'])
df['is_new_client'] = (df['num_years_antig'] <= 3).astype(int)
df['is_loyal_client'] = (df['num_years_antig'] >= 6).astype(int)

print(f"Created temporal features")

# 6. CHANNEL AND ORIGIN FEATURES
print("\nCREATING CHANNEL AND ORIGIN FEATURES\n")

channel_churn_rate = df.groupby('channel_sales')['churn'].mean().to_dict()
df['channel_churn_rate'] = df['channel_sales'].map(channel_churn_rate)
origin_churn_rate = df.groupby('origin_up')['churn'].mean().to_dict()
df['origin_churn_rate'] = df['origin_up'].map(origin_churn_rate)

df['is_missing_channel'] = (df['channel_sales'] == 'MISSING').astype(int)
df['is_missing_origin'] = (df['origin_up'] == 'MISSING').astype(int)

df['is_high_risk_channel'] = (df['channel_churn_rate'] > 0.10).astype(int)
df['is_high_risk_origin'] = (df['origin_churn_rate'] > 0.10).astype(int)

print(f"Created channel and origin features")

# 7. PRODUCT AND SERVICE FEATURES
print("\nCREATING PRODUCT AND SERVICE FEATURES\n")

df['has_multiple_products'] = (df['nb_prod_act'] > 1).astype(int)
df['product_diversity'] = df['nb_prod_act']

df['has_gas_binary'] = (df['has_gas'] == 't').astype(int)
df['electricity_only'] = ((df['has_gas'] == 'f') & (df['cons_12m'] > 0)).astype(int)
df['gas_only'] = ((df['has_gas'] == 't') & (df['cons_gas_12m'] > 0) & (df['cons_12m'] == 0)).astype(int)
df['dual_service'] = ((df['cons_12m'] > 0) & (df['cons_gas_12m'] > 0)).astype(int)

df['power_category'] = pd.cut(df['pow_max'], bins=[0, 10, 15, 25, 400], labels=['low', 'medium', 'high', 'very_high'])
df['is_high_power_client'] = (df['pow_max'] > df['pow_max'].quantile(0.75)).astype(int)

print(f"Created product and service features")

# 8. INTERACTION FEATURES
print("\nCREATING INTERACTION FEATURES\n")

df['price_sensitivity_score'] = df['margin_per_kwh'] * df['is_low_consumer']
df['high_margin_low_consumption'] = ((df['is_high_margin_client'] == 1) & (df['is_low_consumer'] == 1)).astype(int)
df['loyal_low_consumer'] = ((df['is_loyal_client'] == 1) & (df['is_low_consumer'] == 1)).astype(int)
df['new_high_margin'] = ((df['is_new_client'] == 1) & (df['is_high_margin_client'] == 1)).astype(int)
df['risky_channel_low_consumption'] = ((df['is_high_risk_channel'] == 1) & (df['is_low_consumer'] == 1)).astype(int)
df['forecast_error_abs'] = abs(df['actual_vs_forecast'] - 1)
df['is_forecast_accurate'] = (df['forecast_error_abs'] < 0.2).astype(int)

print(f"Created interaction features")

# 9. AGGREGATE AND STATISTICAL FEATURES
print("\nCREATING AGGREGATE FEATURES\n")

df['cons_12m_zscore'] = stats.zscore(df['cons_12m'])
df['net_margin_zscore'] = stats.zscore(df['net_margin'])
df['pow_max_zscore'] = stats.zscore(df['pow_max'])
df['cons_percentile'] = df['cons_12m'].rank(pct=True)
df['margin_percentile'] = df['net_margin'].rank(pct=True)
df['tenure_percentile'] = df['num_years_antig'].rank(pct=True)

print(f"Created aggregate features")

# 10. MERGE ALL PRICE FEATURES
print("\nMERGING PRICE FEATURES WITH CLIENT DATA\n")

price_features = diff[['id', 'offpeak_diff_dec_jan_energy', 'offpeak_diff_dec_jan_power', 
                       'peak_diff_dec_jan_energy', 'peak_diff_dec_jan_power']]
price_features = price_features.merge(price_stats, on='id', how='left')
price_features = price_features.merge(price_trend[['id', 'price_trend_off_peak_var', 
                                                    'price_trend_peak_var', 'price_trend_off_peak_fix']], 
                                      on='id', how='left')
price_features = price_features.merge(price_changes, on='id', how='left')

df_final = df.merge(price_features, on='id', how='left')
print(f"Final dataset shape: {df_final.shape}")

# 11. HANDLE MISSING VALUES IN NEW FEATURES
print("\nHANDLING MISSING VALUES IN NEW FEATURES\n")

price_cols = [col for col in df_final.columns if 'price' in col.lower() or 'peak' in col.lower()]
for col in price_cols:
    if df_final[col].isna().sum() > 0:
        df_final[col].fillna(df_final[col].median(), inplace=True)

df_final = df_final.replace([np.inf, -np.inf], np.nan)
for col in df_final.select_dtypes(include=[np.number]).columns:
    if df_final[col].isna().sum() > 0:
        df_final[col].fillna(df_final[col].median(), inplace=True)

print("Missing values handled")

# 12. REMOVE REDUNDANT/USELESS COLUMNS
print("\nREMOVING REDUNDANT COLUMNS\n")

columns_to_drop = ['id', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
df_final_clean = df_final.drop(columns=columns_to_drop, errors='ignore')

if 'tenure_category' in df_final_clean.columns:
    tenure_map = {'new': 0, 'established': 1, 'loyal': 2, 'veteran': 3}
    df_final_clean['tenure_category_encoded'] = df_final_clean['tenure_category'].map(tenure_map)
    df_final_clean = df_final_clean.drop('tenure_category', axis=1)

if 'power_category' in df_final_clean.columns:
    power_map = {'low': 0, 'medium': 1, 'high': 2, 'very_high': 3}
    df_final_clean['power_category_encoded'] = df_final_clean['power_category'].map(power_map)
    df_final_clean = df_final_clean.drop('power_category', axis=1)

print(f"Final cleaned dataset shape: {df_final_clean.shape}")

# 13. SAVE ENGINEERED DATASET
print("\nSAVING ENGINEERED DATASET\n")

df_final_clean.to_csv('client_data_engineered.csv', index=False)
print("Saved to: client_data_engineered.csv")

# 14. SUMMARY OF FEATURES CREATED
print("\nFEATURE ENGINEERING SUMMARY\n")

new_features = set(df_final_clean.columns) - set(df.columns)
print(f"Total new features created: {len(new_features)}")
print("Feature categories:")
print("  - Price-based features: ~30")
print("  - Consumption features: 7")
print("  - Margin features: 6")
print("  - Temporal features: 16")
print("  - Channel/Origin features: 6")
print("  - Product/Service features: 8")
print("  - Interaction features: 7")
print("  - Statistical features: 6")
print(f"Total columns in final dataset: {df_final_clean.shape[1]}")
print(f"Total rows: {df_final_clean.shape[0]}")

sample_new_features = list(new_features)[:10]
print(df_final_clean[sample_new_features].head())

print("\nFEATURE ENGINEERING COMPLETE!\n")



Loading data...
Client data shape: (14606, 44)
Price data shape: (193002, 8)

CREATING PRICE-BASED FEATURES



C:\Users\USER\AppData\Local\Temp\ipykernel_5132\2016977982.py:94: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  price_changes = price_df.groupby('id').apply(


Created price-based features

CREATING CONSUMPTION-BASED FEATURES

Created consumption-based features

CREATING MARGIN AND PROFITABILITY FEATURES

Created margin and profitability features

CREATING TEMPORAL FEATURES

Created temporal features

CREATING CHANNEL AND ORIGIN FEATURES

Created channel and origin features

CREATING PRODUCT AND SERVICE FEATURES

Created product and service features

CREATING INTERACTION FEATURES

Created interaction features

CREATING AGGREGATE FEATURES

Created aggregate features

MERGING PRICE FEATURES WITH CLIENT DATA

Final dataset shape: (14606, 127)

HANDLING MISSING VALUES IN NEW FEATURES

Missing values handled

REMOVING REDUNDANT COLUMNS

Final cleaned dataset shape: (14606, 122)

SAVING ENGINEERED DATASET

Saved to: client_data_engineered.csv

FEATURE ENGINEERING SUMMARY

Total new features created: 29
Feature categories:
  - Price-based features: ~30
  - Consumption features: 7
  - Margin features: 6
  - Temporal features: 16
  - Channel/Origin fe